In [22]:
import pandas as pd
colnames=['video_id', 'middle_frame_timestamp', 'x1', 'x2', 'y1', 'y2', 'action_id', 'person_id'] 
kinetics_train = pd.read_csv('/data1/hongn/ava_kinetics_v1_0/kinetics_train_v1.0.csv', names=colnames, header=None)

In [17]:
len(kinetics_train['video_id'].unique())

136842

## AVA

In [1]:
import pandas as pd
colnames=['video_id', 'middle_frame_timestamp', 'x1', 'x2', 'y1', 'y2', 'action_id', 'person_id'] 
val_ava = pd.read_csv('/data1/hongn/ava_kinetics_v1_0/ava_val_v2.2.csv', names=colnames, header=None).dropna()
val_ava.head()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from timesformer.datasets.ava import Ava
from timesformer.utils.parser import load_config


import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--cfg_file', help="restore checkpoint")
parser.add_argument('--opts', help="restore checkpoint")
args = parser.parse_args(['--cfg_file', "/data2/hongn/TimePSFormer/configs/Kinetics/SLOWFAST_8x8_R101.yaml"]) 
cfg = load_config(args)

dataset = Ava(cfg, "train")

/data2/hongn/miniconda3/envs/moose/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
imgs, label_arrs, idx, _, extra_data = dataset[4]

In [14]:
label_arrs.shape

(5, 400)

In [17]:
len(out[1])

6

## SSv2 Everything

In [20]:
import glob
import os
ssv2_dir = glob.glob('/data1/hongn/SSv2/frames/*')
list_empty_folder = []
for path in ssv2_dir:
    if len(os.listdir(path)) == 0:
        list_empty_folder.append(path)

In [ ]:
['/data1/hongn/SSv2/frames/123178', '/data1/hongn/SSv2/frames/123156']
list_empty_folder

['/data1/hongn/SSv2/frames/123178', '/data1/hongn/SSv2/frames/123156']

In [66]:
#TimeSFormer AVA lavel, turn out, it does not need to be processed
import pandas as pd
val = pd.read_csv('/data1/hongn/SSv2/labels/val.csv')
col_merge = 'original_vido_id video_id frame_id path labels'
val[col_merge.split(' ')] = val[col_merge].str.split(' ', expand=True)
val = val.drop(col_merge, axis=1)
# val.to_csv('/data1/hongn/SSv2/labels/train.csv', index=False, header=True)
# val.head()

In [67]:
from tqdm import tqdm

list_empty_folder = []

for name in tqdm(val.path):
    if(os.path.isfile('/data1/hongn/SSv2/frames/' + name) == False):
        list_empty_folder.append(name)

100%|██████████| 3018451/3018451 [01:19<00:00, 37919.14it/s] 


In [ ]:
list_empty_folder

In [68]:
folder_redo = [name.split('/')[0] for name in list_empty_folder]
set(folder_redo)

set()

## HAA500

In [1]:
import pandas as pd
train = pd.read_csv('/data1/hongn/kinetics-dataset/k400/train.csv')
train.head()

,/data1/hongn/kinetics-dataset/k400/train/8mbN0ubh0q8_000000_000010.mp4,101
0,/data1/hongn/kinetics-dataset/k400/train/9bfc-...,17
1,/data1/hongn/kinetics-dataset/k400/train/VV0KA...,167
2,/data1/hongn/kinetics-dataset/k400/train/sWCnx...,318
3,/data1/hongn/kinetics-dataset/k400/train/Am6c9...,356
4,/data1/hongn/kinetics-dataset/k400/train/pC1hX...,231


In [3]:
train_haa500 = pd.read_csv('/data1/hongn/haa500/raw/climb_icecliff.txt')
train_haa500.head()

,https://www.youtube.com/watch?v=Uo_sDYOvW98,113.19,116.11,1,1.1
0,https://www.youtube.com/watch?v=79s5BD0301o,348.50,351.50,1,1
1,https://www.youtube.com/watch?v=1syox_mjE6s,124.41,127.13,0,1
2,https://www.youtube.com/watch?v=OnV0XbwSAAM,423.48,426.39,0,2
3,https://www.youtube.com/watch?v=Xp83UFuKxyE,616.31,619.07,1,1
4,https://www.youtube.com/watch?v=-iDBPhSc2mU,130.50,133.15,0,1


In [85]:
from glob import glob
import numpy as np

action_folders = glob('/data1/hongn/haa500/video/*')
label_nameidx_map = pd.DataFrame(columns=['label_name', 'label_index'])
# all_data = pd.DataFrame(columns=['paths', 'labels'])
# actions
action_paths = []
labels = []
for i in range(len(action_folders)):
    action_name = action_folders[i].split('/')[-1]
    # print(action_name)
    label_nameidx_map.loc[i] = [action_name, i]
    actioni_paths = glob(f'/data1/hongn/haa500/video/{action_name}/*')
    action_paths.append(actioni_paths)
    labels.append(np.ones(len(actioni_paths)) * i)

action_paths = np.array(action_paths).flatten()
labels = np.array(labels).flatten().astype(int)

In [86]:
from sklearn.model_selection import train_test_split
X_trainval, X_test, y_trainval, y_test = train_test_split(action_paths, labels, test_size=0.33, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.33, random_state=42, shuffle=True)

In [95]:
train = pd.DataFrame(np.stack([X_train, y_train], axis=1), columns=['paths', 'labels'])
val = pd.DataFrame(np.stack([X_val, y_val], axis=1), columns=['paths', 'labels'])
test = pd.DataFrame(np.stack([X_test, y_test], axis=1), columns=['paths', 'labels'])

label_nameidx_map.to_csv('/data1/hongn/haa500/label_nameidx_map.csv',index=False, header=False)
train.to_csv('/data1/hongn/haa500/train.csv',index=False, header=False)
val.to_csv('/data1/hongn/haa500/val.csv',index=False, header=False)
test.to_csv('/data1/hongn/haa500/test.csv',index=False, header=False)

In [90]:
np.stack([X_train, y_train], axis=1)

array([['/data1/hongn/haa500/video/basketball_shoot/basketball_shoot_007.mp4',
        '121'],
       ['/data1/hongn/haa500/video/washing_dishes/washing_dishes_000.mp4',
        '297'],
       ['/data1/hongn/haa500/video/snowboard_jump/snowboard_jump_010.mp4',
        '159'],
       ...,
       ['/data1/hongn/haa500/video/one_arm_push_up/one_arm_push_up_015.mp4',
        '155'],
       ['/data1/hongn/haa500/video/sprint_run/sprint_run_006.mp4', '395'],
       ['/data1/hongn/haa500/video/frisbee_catch/frisbee_catch_014.mp4',
        '432']], dtype='<U109')

In [88]:
y_train

array([121, 297, 159, ..., 155, 395, 432])

In [81]:
labels.astype(int)

array([  0,   0,   0, ..., 499, 499, 499])

In [69]:
import numpy as np
actioni = glob('/data1/hongn/haa500/video/gym_plank/*')
len(actioni)
np.ones(len(actioni)) * i

array([499., 499., 499., 499., 499., 499., 499., 499., 499., 499., 499.,
       499., 499., 499., 499., 499., 499., 499., 499., 499.])

In [68]:
all_data = pd.DataFrame(columns=['paths', 'labels'])
temp = np.stack([actioni, np.ones(len(actioni))], axis=1)
all_data = pd.concat([all_data,pd.Series(temp)], ignore_index=True)
all_data

ValueError: Data must be 1-dimensional, got ndarray of shape (20, 2) instead

In [7]:
actions[12]

'/data1/hongn/haa500/raw/play_ukulele.txt'